In [1]:
import tensorflow as tf

In [2]:
!pip install -q -U keras-tuner

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

X_train = X_train / 255.0
X_test = X_test / 255.0

The code below defines the layers and the hyperparameters to be tuned, as well as the parameters for tuning the hyperparameters.

In [4]:
print(X_train.shape)

(50000, 32, 32, 3)


In [5]:
alt_model = False

In [6]:
if alt_model == False:
  def model_builder(hp):
      model = tf.keras.Sequential()


      hp_initial_learning_rate = hp.Choice('learning_rate',
                                           values=[0.01, 0.001, 0.0001])
      hp_decay_rate = hp.Float('decay_rate', min_value=0.9, max_value=0.98,
                             step=0.01)
      lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        hp_initial_learning_rate,
        decay_steps=100000,
        decay_rate=hp_decay_rate,
        staircase=True)

      #Conv
      hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
      hp_kernel_size = hp.Int('kernel_size', min_value=1, max_value=6, step=1)
      hp_filters = hp.Int('filters', min_value=1, max_value=64, step = 8)

      model.add(tf.keras.layers.Conv2D(filters=hp_filters,
                                       kernel_size=(hp_kernel_size, hp_kernel_size),
                                       activation=hp_activation,
                                       input_shape=(32, 32, 3)))
      model.add(tf.keras.layers.Flatten())


      #Dense

      hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=1000, step=100)
      hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=1000, step=100)


      model.add(tf.keras.layers.Dense(units=hp_layer_1, activation=hp_activation))
      model.add(tf.keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
      model.add(tf.keras.layers.Dense(10, activation='softmax'))

      model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                    metrics=['accuracy'])

      return model


In [7]:
if alt_model:
  def model_builder(hp):
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))

      model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                    metrics=['accuracy'])


      return model


In [8]:
import keras_tuner as kt

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='dir',
                     project_name='mnist')

Reloading Tuner from dir/mnist/tuner0.json


In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
#this will stop the training if the model's val_loss does not improve over 3
#consecutive epochs

Conv layers kernel is fixed: 3x3

In [ ]:
tuner.search(X_train, y_train, epochs=20, batch_size=200, validation_split=0.2, callbacks=[stop_early])


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
relu              |relu              |activation
301               |301               |layer_1
201               |201               |layer_2
0.001             |0.001             |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
200/200 [==============================] - 22s 71ms/step - loss: 2.0727 - accuracy: 0.2455 - val_loss: 1.9487 - val_accuracy: 0.3041
Epoch 2/2
110/200 [===============>..............] - ETA: 2s - loss: 1.9065 - accuracy: 0.3237

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
model = tuner.hypermodel.build(best_hps)

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2,
                    callbacks=[stop_early])